In [1]:
import torch, torch.nn as nn
from torchviz import make_dot
from models import MLP

In [2]:
# Initialize main model
main_mapper = MLP(2,5,[12,13], ['relu','relu'], technique='delta') # Total weights = 2*12 + 12*13 + 13*5 = 245 
print(f"------MAIN MAPPER-------")
print(f"{main_mapper}")

# Hypernetwork

hnn = MLP(1,245,[100],['relu'])
print(f"------Hypernetwork-------")
print(f"{hnn}")

------MAIN MAPPER-------
MLP(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=13, bias=True)
    (3): ReLU()
    (4): Linear(in_features=13, out_features=5, bias=True)
  )
)
------Hypernetwork-------
MLP(
  (layers): ModuleList(
    (0): Linear(in_features=1, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=245, bias=True)
  )
)


In [3]:
# # Add forward hooks to hypernetwork 
# hnn.layers[0].register_forward_hook(lambda module, i, output: print('fc1 forward hook called'))
# hnn.layers[2].register_forward_hook(lambda module, i, output: print('fc2 forward hook called'))
# main_mapper.layers[0].register_forward_hook(lambda module, i, output: print('fc1 forward hook called'))
# # Register backward hooks
# hnn.layers[0].register_full_backward_hook(lambda module, grad_i, grad_output: print(f'fc1 backward hook called'))
# hnn.layers[2].register_full_backward_hook(lambda module, grad_i, grad_output: print(f'fc2 backward hook called'))

In [4]:
weights = {k:v for k,v in main_mapper.state_dict().items() if 'weight' in k}

In [5]:
# hypernetwork forward pass
u = torch.randn(1, requires_grad= True)
hnn.technique='none'
w = hnn(u)
print(f"Weights shape is: {w.shape}")

# reshape hypernetwork weights and update weights dict
slide = 0
for k,v in weights.items():
    weights[k] = w[slide: slide+v.numel()].reshape(v.shape)
    slide += v.numel() 

Weights shape is: torch.Size([245])


In [6]:
x = torch.randn(2)
y = main_mapper(x, weights)

In [7]:
hnn.zero_grad()
make_dot(y.mean(), params=dict(hnn.named_parameters())).render("backward_hnn", format="png")

'backward_hnn.png'

In [3]:
import torch

x = torch.randn((2,3))
y = torch.Tensor([1])
print(x, y)
print(x+y)

tensor([[ 1.0056, -0.1970, -0.1704],
        [ 0.8398,  0.5012,  0.8547]]) tensor([1.])
tensor([[2.0056, 0.8030, 0.8296],
        [1.8398, 1.5012, 1.8547]])
